In [ ]:
import cv2

video_path = '04_01.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video file.")
else:
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("Frames Per Second (fps):", fps)
    print("Frame Count:", frame_count)
    cap.release()


In [ ]:
import cv2
import numpy as np
video_path = '04_01.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frames = []

new_frame = {}
old_frame = {}

lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
)
displacements = []

for frame_index in range(frame_count):
    ret, frame = cap.read()  # Read a frame

    if not ret:
        break  # Break the loop if the video ends
    
    old_frame = new_frame
    new_frame = frame
    if(frame_index == 0):
        continue
    
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    new_gray = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)
    
    frames.append(frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, thresholded = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_list = []
    contours_list.append(contours)
    
    min_contour_area = 50
    contours_arr = []
    contours_area = []

    for contour in contours:
        contours_arr.append(contour)
        contours_area.append(cv2.contourArea(contour))

    filtered_contours = [contour for contour in contours_arr if cv2.contourArea(contour) > min_contour_area]
    
    target_points = []

    for contour in filtered_contours:
        moments = cv2.moments(contour)
        center_x = moments['m10'] / moments['m00']
        center_y = moments['m01'] / moments['m00']
        target_points.append([center_x, center_y])
        
    target_points = np.array(target_points, dtype=np.float32)
    
    new_points, status, _ = cv2.calcOpticalFlowPyrLK(old_gray, new_gray, target_points, None, **lk_params)
    
    diff = []
    goodNew = []
    goodOld = []
    for i in range(len(status)):
        if status[i] == 1:
            goodNew.append(new_points[i])
            goodOld.append(target_points[i])  
    
    for i in range(len(goodNew)):
        if(status[i]):
            diff.append([goodNew[i][0] - goodOld[i][0], goodNew[i][1] - goodOld[i][1]])

    meanX = np.mean([point[0] for point in diff])
    meanY = np.mean([point[1] for point in diff])

    mean_displacement = { "x": meanX, "y": meanY}
    displacements.append(mean_displacement)

print(frame_count)
cap.release()

In [ ]:
def sampler_and_writer(period, filename, displacement):
    # 7 frames sample period
    sampled_list = displacement[1::period]
    print(sampled_list)
    with open(filename, "w") as f:
        f.write("export const directions = [\n")
        for item in sampled_list:
            print(item)
            f.write("    {\n")    
            f.write(f"        \"x\": {item['x']},\n")
            f.write(f"        \"y\": {item['y']},\n")
            f.write("    },\n")
        f.write("]\n") 
        f.close()    

sampler_and_writer(7, "directions.js", displacements)